In [8]:
import os
import json
import pandas as pd
import traceback

In [17]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
KEY = os.getenv("OPENAI_API_KEY")

In [20]:
llm = ChatOpenAI(openai_api_key=key, model_name= "gpt-3.5-turbo", temperature=0.3)

In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [22]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}

In [62]:
oTEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [63]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
{response_json}

"""

In [64]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [65]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [66]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [67]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [68]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)



In [69]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [111]:
file_path=r"/Users/harishgurram/Documents/GenAI/mcqgen/data.txt"


In [112]:
file_path

'/Users/harishgurram/Documents/GenAI/mcqgen/data.txt'

In [113]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [114]:
print(TEXT)


In the most general of terms, music is the arrangement of sound to create some combination of form, harmony, melody, rhythm, or otherwise expressive content.[1][2][3] Definitions of music vary depending on culture,[4] though it is an aspect of all human societies and a cultural universal.[5] While scholars agree that music is defined by a few specific elements, there is no consensus on their precise definitions.[6] The creation of music is commonly divided into musical composition, musical improvisation, and musical performance,[7] though the topic itself extends into academic disciplines, criticism, philosophy, psychology, and therapeutic contexts. Music may be performed using a vast range of instruments, including the human voice to sing, and thus is often credited for its extreme versatility and opportunity for creativity.[8]

In some musical contexts, a performance or composition may be to some extent improvised. For instance, in Hindustani classical music, the performer plays spon

In [115]:
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [116]:
NUMBER=3
SUBJECT="music"
TONE="simple"

In [117]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:In the most general of terms, music is the arrangement of sound to create some combination of form, harmony, melody, rhythm, or otherwise expressive content.[1][2][3] Definitions of music vary depending on culture,[4] though it is an aspect of all human societies and a cultural universal.[5] While scholars agree that music is defined by a few specific elements, there is no consensus on their precise definitions.[6] The creation of music is commonly divided into musical composition, musical improvisation, and musical performance,[7] though the topic itself extends into academic disciplines, criticism, philosophy, psychology, and therapeutic contexts. Music may be performed using a vast range of instruments, including the human voice to sing, and thus is often credited for its extreme versatility and opportunity for creativity.[8]

In some musical contexts, a performance or composi

In [118]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:10562
Prompt Tokens:10092
Completion Tokens:470
Total Cost:0.016078000000000002


In [119]:
response

{'text': 'In the most general of terms, music is the arrangement of sound to create some combination of form, harmony, melody, rhythm, or otherwise expressive content.[1][2][3] Definitions of music vary depending on culture,[4] though it is an aspect of all human societies and a cultural universal.[5] While scholars agree that music is defined by a few specific elements, there is no consensus on their precise definitions.[6] The creation of music is commonly divided into musical composition, musical improvisation, and musical performance,[7] though the topic itself extends into academic disciplines, criticism, philosophy, psychology, and therapeutic contexts. Music may be performed using a vast range of instruments, including the human voice to sing, and thus is often credited for its extreme versatility and opportunity for creativity.[8]\n\nIn some musical contexts, a performance or composition may be to some extent improvised. For instance, in Hindustani classical music, the performe

In [120]:
quiz =response.get("quiz")


In [121]:
quiz=json.loads(quiz)

In [122]:
quiz

{'1': {'mcq': 'Which era of music aimed to imitate the art and philosophy of Ancient Greece and Rome?',
  'options': {'a': 'Baroque',
   'b': 'Classicism',
   'c': 'Romanticism',
   'd': 'Renaissance'},
  'correct': 'b'},
 '2': {'mcq': 'Which composer is known for his Cello suites during the Baroque era?',
  'options': {'a': 'Ludwig van Beethoven',
   'b': 'Antonio Vivaldi',
   'c': 'Johann Sebastian Bach',
   'd': 'George Frideric Handel'},
  'correct': 'c'},
 '3': {'mcq': 'Which Asian country is the home of the gamelan ensemble?',
  'options': {'a': 'India', 'b': 'China', 'c': 'Indonesia', 'd': 'Japan'},
  'correct': 'c'}}

In [123]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [124]:
quiz_table_data

[{'MCQ': 'Which era of music aimed to imitate the art and philosophy of Ancient Greece and Rome?',
  'Choices': 'a: Baroque | b: Classicism | c: Romanticism | d: Renaissance',
  'Correct': 'b'},
 {'MCQ': 'Which composer is known for his Cello suites during the Baroque era?',
  'Choices': 'a: Ludwig van Beethoven | b: Antonio Vivaldi | c: Johann Sebastian Bach | d: George Frideric Handel',
  'Correct': 'c'},
 {'MCQ': 'Which Asian country is the home of the gamelan ensemble?',
  'Choices': 'a: India | b: China | c: Indonesia | d: Japan',
  'Correct': 'c'}]

In [125]:
quiz=pd.DataFrame(quiz_table_data)


In [126]:
quiz

,MCQ,Choices,Correct
0,Which era of music aimed to imitate the art an...,a: Baroque | b: Classicism | c: Romanticism | ...,b
1,Which composer is known for his Cello suites d...,a: Ludwig van Beethoven | b: Antonio Vivaldi |...,c
2,Which Asian country is the home of the gamelan...,a: India | b: China | c: Indonesia | d: Japan,c


In [127]:
quiz.to_csv("music.csv",index=False)


In [3]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_06_2024_12_03_40'

In [2]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_06_2024_12_03_36'